In [1]:
!pip install -e ../../

Obtaining file:///C:/Users/netxph/Projects/mlpouch
  Attempting uninstall: mlpouch
    Found existing installation: mlpouch 0.0.1
    Uninstalling mlpouch-0.0.1:
      Successfully uninstalled mlpouch-0.0.1
  Running setup.py develop for mlpouch


In [2]:
import pandas as pd
from mlpouch.decomposition import FunkSVD
import logging

logging.basicConfig(level=logging.DEBUG)

In [3]:
movies = pd.read_csv("data/movies_clean.csv")
del movies['Unnamed: 0']
movies.head()

,movie_id,movie,genre,date,1800's,1900's,2000's,History,News,Horror,...,Fantasy,Romance,Game-Show,Action,Documentary,Animation,Comedy,Short,Western,Thriller
0,8,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
1,10,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,12,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,91,Le manoir du diable (1896),Short|Horror,1896,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [4]:
reviews = pd.read_csv("data/reviews_clean.csv")
del reviews['Unnamed: 0']
reviews.head()

,user_id,movie_id,rating,timestamp,date,month_1,month_2,month_3,month_4,month_5,...,month_9,month_10,month_11,month_12,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
0,1,68646,10,1381620027,2013-10-12 23:20:27,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
1,1,113277,10,1379466669,2013-09-18 01:11:09,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,422720,8,1412178746,2014-10-01 15:52:26,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
3,2,454876,8,1394818630,2014-03-14 17:37:10,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2,790636,7,1389963947,2014-01-17 13:05:47,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [5]:
def create_train_test(X, order_by, training_size, testing_size, sample_size=10000):

    X = X.sort_values(by=order_by)
    sample = X.head(sample_size)
    
    X_train = sample.head(training_size)
    X_test = sample.tail(testing_size)
    
    return X_train, X_test

In [6]:
reviews_train, reviews_sub = create_train_test(reviews, 'date', 8000, 2000)

In [7]:
reviews_sub = reviews_train[["user_id", "movie_id", "rating"]]
print(reviews_sub.shape)
reviews_sub.head()

(8000, 3)


,user_id,movie_id,rating
498923,37287,2171847,6
442554,33140,444778,8
81920,6338,1411238,6
584570,43691,1496422,7
450669,33799,118799,5


In [8]:
movie_reviews = reviews_sub.groupby(['user_id', 'movie_id'])['rating'].max().unstack()
movie_reviews.head()

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
dec = FunkSVD(iters=20, learning_rate=0.005)
dec.fit(movie_reviews)

DEBUG:root:Optimization Statistics
DEBUG:root:Iterations | Mean Squared Error 
DEBUG:root:1 		 32.48930819574863
DEBUG:root:2 		 21.146990918133486
DEBUG:root:3 		 15.80485580330642
DEBUG:root:4 		 12.366643382187073
DEBUG:root:5 		 9.93815953811539
DEBUG:root:6 		 8.141522622486045
DEBUG:root:7 		 6.7597938712790935
DEBUG:root:8 		 5.663965146043592
DEBUG:root:9 		 4.775584321425906
DEBUG:root:10 		 4.045132168324753
DEBUG:root:11 		 3.439597610970296
DEBUG:root:12 		 2.935509159708285
DEBUG:root:13 		 2.5151130838615963
DEBUG:root:14 		 2.1642994753314944
DEBUG:root:15 		 1.8714805748221077
DEBUG:root:16 		 1.626973055089985
DEBUG:root:17 		 1.4226328920638622
DEBUG:root:18 		 1.2516097970734437
DEBUG:root:19 		 1.108158292553491
DEBUG:root:20 		 0.9874794399596041


FunkSVD(iters=20, learning_rate=0.005)

In [10]:
dec.X_transformed

movie_id,2844,8133,13427,14142,14538,14872,15163,15324,15400,17925,...,2625030,2635622,2638984,2645104,2645164,2646378,2649128,2735466,2748368,2750600
user_id,,,,,,,,,,,,,,,,,,,,,
8,6.421525,6.911593,3.795180,6.337280,8.232057,6.169347,8.044525,7.737060,7.022000,6.108847,...,7.313559,7.748660,6.447472,5.989908,2.439664,7.569097,9.873356,5.197228,8.242771,7.053455
46,6.282898,6.320200,3.374702,6.741297,7.924548,6.298771,7.426876,7.559652,7.081787,6.010205,...,6.727501,7.849895,7.512109,6.498497,2.468388,7.755725,9.838724,6.290759,6.796005,6.092603
48,5.196122,5.085814,3.284703,5.112643,6.790740,5.480601,6.654335,6.750501,5.838324,5.178977,...,5.663363,6.648324,5.943247,4.865162,1.936347,6.049554,8.374546,4.387880,5.478726,5.531038
51,5.412415,5.887220,2.623709,5.827260,6.506569,5.512671,5.746343,6.119834,5.917633,4.971428,...,5.556395,6.502974,6.065043,5.866152,2.481158,7.037820,7.653174,5.065052,6.522303,5.019440
66,5.815430,4.836671,3.776925,5.953303,7.669240,6.975379,7.333703,8.185953,6.824196,6.114559,...,5.730248,8.063541,8.181508,5.814444,2.214355,6.888281,9.862267,5.666854,3.948411,5.520976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53856,5.660360,5.103226,3.104336,6.368705,7.404681,5.516252,7.176548,7.084077,6.663067,5.589641,...,6.213653,7.389686,7.663732,5.886325,1.849277,6.781777,9.820739,6.727128,5.060451,5.376172
53932,6.013434,6.003825,2.859635,6.845841,7.388926,6.042125,6.627746,6.973988,6.837554,5.650863,...,6.192310,7.515227,7.698494,6.711165,2.494410,7.760617,9.257626,6.748522,6.172298,5.285886
53943,5.317415,4.771717,2.703751,6.204290,6.838552,5.203323,6.453770,6.506136,6.290311,5.192463,...,5.685171,6.943068,7.488026,5.814855,1.822038,6.570433,9.118126,6.720776,4.562454,4.732673


In [11]:
user_ids = [8, 46, 48, 51, 66]
dec.recommend(user_ids)